In [16]:
# --- Celda 1: Instalaciones ---
# Instalamos Gradio (interfaz web), sentence-transformers (embeddings),
# faiss-cpu (búsqueda vectorial) y la librería de Google (para Gemini).
# --- Celda 1: Instalaciones ---
print("Instalando librerías...")
!pip install gradio sentence-transformers faiss-cpu google-generativeai python-dotenv --quiet
print("✅ Librerías instaladas.")

Instalando librerías...
✅ Librerías instaladas.


In [17]:
# --- Celda 2: Importaciones ---
import gradio as gr
import json
import numpy as np
import faiss
from pathlib import Path
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import os                       # 👈 Importante para dotenv
from dotenv import load_dotenv  # 👈 Importamos la función
print("✅ Librerías importadas.")

✅ Librerías importadas.


In [18]:
# --- Celda 3: Configuración y API Key (con DotEnv) ---

# 1. Cargar las variables del archivo .env a la memoria del sistema
load_dotenv()

# 2. Leer la API key desde las variables de entorno
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    print("🛑 ¡ERROR! No se encontró 'GEMINI_API_KEY'.")
    print("Asegúrate de haber creado el archivo .env y puesto la clave allí.")
else:
    try:
        genai.configure(api_key=API_KEY)
        print("✅ API Key de Gemini cargada de forma segura desde .env.")
    except Exception as e:
        print(f"🛑 Error configurando la API Key: {e}")

# Nombres de archivos y modelos
DATA_FILE = "music_faiss_map_with_embeddings.json"
EMBED_MODEL = "all-MiniLM-L6-v2"
GEMINI_MODEL = "models/gemini-2.5-flash"

✅ API Key de Gemini cargada de forma segura desde .env.


In [19]:
# --- Celda 4: Cargar Datos y Construir el "Cerebro" (FAISS) ---

p = Path(DATA_FILE)
if not p.exists():
    print(f"❌ ERROR: No se encontró el archivo '{DATA_FILE}'.")
    print("Por favor, sube tu archivo JSON al panel de Archivos (a la izquierda).")
else:
    print(f"Cargando datos desde {DATA_FILE}...")
    data = json.loads(p.read_text(encoding="utf-8"))
    metas = data["metadatas"]
    print(f"✅ Canciones cargadas: {len(metas)}")

    # Cargar el modelo de embeddings
    print(f"Cargando modelo de embeddings '{EMBED_MODEL}'...")
    model = SentenceTransformer(EMBED_MODEL)

    # Extraer embeddings y doc_ids para FAISS
    embeddings = []
    doc_ids = []
    for doc_id, meta in metas.items():
        emb = meta.get("embedding")
        if emb:
            embeddings.append(np.array(emb, dtype=np.float32))
            doc_ids.append(doc_id)

    # Construir el índice FAISS
    emb_matrix = np.vstack(embeddings)
    faiss.normalize_L2(emb_matrix) # Normalizar para usar producto interno (IP)

    dim = emb_matrix.shape[1]
    index = faiss.IndexFlatIP(dim) # IP = Inner Product (producto escalar)
    index.add(emb_matrix)

    print(f"✅ Índice FAISS (el 'cerebro' de búsqueda) creado con {index.ntotal} canciones.")

Cargando datos desde music_faiss_map_with_embeddings.json...
✅ Canciones cargadas: 50
Cargando modelo de embeddings 'all-MiniLM-L6-v2'...
✅ Índice FAISS (el 'cerebro' de búsqueda) creado con 50 canciones.


In [20]:
# --- Celda 5: Funciones del RAG (Las "Herramientas") ---

def embed_query(query):
    """Convierte una consulta de texto en un vector embedding."""
    return model.encode([query], convert_to_numpy=True)[0]

def retrieve_similar(query_emb, k=4):
    """Busca en FAISS los k documentos más similares."""
    q = np.asarray(query_emb, dtype=np.float32).reshape(1, -1)
    faiss.normalize_L2(q)
    
    D, I = index.search(q, k) # D=Distancias (scores), I=Índices
    
    results = []
    for score, idx in zip(D[0], I[0]):
        song_id = doc_ids[idx]
        meta = metas[song_id]
        results.append({
            "doc_id": song_id,
            "score": float(score),
            "meta": meta
        })
    return results

def build_rag_prompt(query, retrieved_docs):
    """Construye el prompt para Gemini con el contexto encontrado."""
    context = ""
    for i, doc in enumerate(retrieved_docs):
        meta = doc["meta"]
        metadata_dict = meta.get('metadata', {})
        context += f"""
[Documento {i+1}]
Título: {meta.get('title', 'N/A')}
Artista: {meta.get('artist', 'N/A')}
Año: {metadata_dict.get('released', 'N/A')}
Género: {metadata_dict.get('genre', 'N/A')}
"""
    
    prompt = f"""
Eres un asistente experto en música llamado 'Discogs RAG'.
Tu tarea es responder la pregunta del usuario basándote *únicamente* en el contexto proporcionado.
Si la respuesta no está en el contexto, di "No encontré información sobre eso en mi base de datos".

---
[CONTEXTO]
{context.strip()}
---

[PREGUNTA DEL USUARIO]
{query}

[TU RESPUESTA]
"""
    return prompt

def call_gemini(prompt_text):
    """Llama a la API de Gemini para generar una respuesta."""
    try:
        model_obj = genai.GenerativeModel(GEMINI_MODEL)
        response = model_obj.generate_content(prompt_text)
        return response.text
    except Exception as e:
        print(f"Error al llamar a Gemini: {e}")
        return "Hubo un error al contactar al modelo de IA. Por favor, revisa tu API Key."

print("✅ Funciones RAG (embed, retrieve, build, call) definidas.")

✅ Funciones RAG (embed, retrieve, build, call) definidas.


In [21]:
# --- Celda 6: Función Principal (El "Orquestador") ---

def responder_pregunta_rag(pregunta_usuario, historial_chat):
    """
    Esta es la función principal que Gradio llamará.
    Recibe la pregunta y el historial, y devuelve la respuesta.
    """
    if not pregunta_usuario.strip():
        return "Por favor, hazme una pregunta."

    print(f"\nPregunta recibida: '{pregunta_usuario}'")
    
    # Paso 1: Embed (Convertir pregunta a vector)
    q_emb = embed_query(pregunta_usuario)
    
    # Paso 2: Retrieve (Buscar en FAISS)
    retrieved_docs = retrieve_similar(q_emb, k=4)
    
    # Paso 3: Build Prompt (Construir prompt con contexto)
    prompt = build_rag_prompt(pregunta_usuario, retrieved_docs)
    print(f"Prompt construido para Gemini:\n{prompt}\n")
    
    # Paso 4: Generate (Llamar a Gemini)
    respuesta = call_gemini(prompt)
    
    print(f"Respuesta generada: '{respuesta}'")
    return respuesta

print("✅ Función 'orquestadora' de Gradio definida.")

✅ Función 'orquestadora' de Gradio definida.


In [ ]:
# --- Celda 7: ¡Lanzar la Interfaz (Gradio)! ---

print("🚀 Lanzando interfaz de Gradio...")

iface = gr.ChatInterface(
    fn=responder_pregunta_rag,
    title="🎵 Experto Musical (RAG + Discogs)",
    description="Hazme una pregunta sobre la base de datos de música. Usaré RAG para encontrar la respuesta.",
    examples=[
        "¿Qué discos tiene jazz?", 
        "Recomiéndame vinilos de rock europeo", 
        "Canciones de rock clásico con guitarras"
    ],
    theme="soft"
)

# .launch(share=True) crea un enlace público temporal (72h)
# para que puedas probarlo en tu navegador o teléfono.
iface.launch(debug=True, share=True)

🚀 Lanzando interfaz de Gradio...


c:\Users\bryan\Desktop\proyectosClase\scraping\Scraping171025\.venv311\Lib\site-packages\gradio\chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.



Pregunta recibida: 'tienes discos de rap'
Prompt construido para Gemini:

Eres un asistente experto en música llamado 'Discogs RAG'.
Tu tarea es responder la pregunta del usuario basándote *únicamente* en el contexto proporcionado.
Si la respuesta no está en el contexto, di "No encontré información sobre eso en mi base de datos".

---
[CONTEXTO]
[Documento 1]
Título: LAX
Artista: The Game (2)
Año: 2008
Género: Hip Hop

[Documento 2]
Título: Life 'N Perspectives Of A Genuine Crossover
Artista: Urban Dance Squad
Año: 2016-11-07
Género: Hip Hop, Rock

[Documento 3]
Título: Kinematografia
Artista: Paktofonika
Año: N/A
Género: Hip Hop

[Documento 4]
Título: Ascenseur Pour L'échafaud * Des Femmes Disparaissent
Artista: The Miles Davis Quintet & Art Blakey & The Jazz Messengers
Año: N/A
Género: Jazz, Stage & Screen
---

[PREGUNTA DEL USUARIO]
tienes discos de rap

[TU RESPUESTA]


Respuesta generada: 'Sí, tengo discos de rap:

*   LAX de The Game (2)
*   Life 'N Perspectives Of A Genuine Cro